# 导包

In [61]:
import re
import numpy as np
import pandas as pd
import os
import xlwt
import xlrd
import openpyxl
import xlsxwriter
import datetime
from xlutils import copy
from openpyxl import load_workbook
from xlrd import open_workbook

In [62]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 参数设置

In [63]:
date = datetime.date.today()
month = str(date.month) + '月'
month

'3月'

<b><font size=1 color=green>特殊时间段：</font></b>

In [64]:
month = '一季度'

# 导入数据

In [85]:
block_dir = '原始数据/' + month + '/'
block_data_file_path = os.listdir(block_dir)

In [86]:
for file in os.listdir(block_dir):
    if '新房' in file:
        new_house_data = pd.read_excel(block_dir + file, dtype=object)
    if '住宅' in file and '非' not in file:
        house_data = pd.read_excel(block_dir + file, dtype=object)
    if '非住宅' in file:
        non_house_data = pd.read_excel(block_dir + file, dtype=object)
    if '全市' in file:
        all_data = pd.read_excel(block_dir + file, dtype=object)

H:\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## 楼市排行榜

### 计算排行

In [87]:
new_house_data = new_house_data.drop(0)[['项目名称', '城区', '成交套数', '成交面积(㎡)', '预测成交总价(万元)', '预测成交均价(元/㎡)']]
new_house_data.columns = ['项目名称', '所属区域', '签约套数', '签约面积（㎡）', '签约金额（万元）', '签约均价（元/㎡）']

new_house_data= new_house_data.replace({'祥生云湖城':'祥生银湖宸语'})
calc_data_new_house = new_house_data.copy()
new_house_data.head()

,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
1,绿城·桂冠东方,萧山区,1162,162629.46,754238,46378
2,前湾国际中心,萧山区,931,130646.62,385604,29515
3,滨江·望涛月明,萧山区,757,91957.07,343850,37392
4,盈都·五角广场,钱塘新区,579,27001.84,39246,14535
5,滨江·滨涛映月,萧山区,566,66296.39,247950,37400


In [88]:
merge_data = calc_data_new_house.groupby(['所属区域', '项目名称'])[['签约套数', '签约面积（㎡）', '签约金额（万元）']].sum()
merge_data.loc[:, '均价'] = merge_data.loc[:, '签约金额（万元）']*10000/merge_data.loc[:, '签约面积（㎡）']
merge_data.loc[:, ['签约面积（㎡）', '均价', '签约金额（万元）']] = round(merge_data.loc[:, ['签约面积（㎡）', '均价', '签约金额（万元）']], 0).astype(int)
merge_data = merge_data.reset_index()
merge_data.rename(columns={'均价':'签约均价（元/㎡）'},inplace=True) 
merge_data=merge_data[['项目名称', '所属区域', '签约套数', '签约面积（㎡）', '签约金额（万元）', '签约均价（元/㎡）']]
merge_data

,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
0,世茂风颂府,上城区,70,9347,37945,40595
1,九天泽一国际,上城区,3,118,791,66926
2,信达东莱府,上城区,1,38,154,40968
3,招商德信·花芸春境,上城区,6,614,1944,31647
4,招商越秀·公园1872,上城区,51,1993,6850,34378
...,...,...,...,...,...,...
573,融创杭臻源,钱塘新区,87,10650,40722,38238
574,融创正荣·杭曜之城,钱塘新区,240,28466,85350,29983
575,金成祥新·潮起钱塘,钱塘新区,1,100,211,21111
576,龙湖·滟澜星座,钱塘新区,1,73,135,18400


In [89]:
merge_data[merge_data['签约均价（元/㎡）']<0]

,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）


In [90]:
# 主城区套数
main_city = ['上城区', '下城区', '江干区', '拱墅区', '西湖区', '滨江区', '钱塘新区']
main_city_quantity = merge_data[merge_data['所属区域'].isin(main_city)].sort_values(by=['签约套数', '签约面积（㎡）'], ascending=False).head(11)
main_city_quantity.insert(0, '排名', list(range(1,12,1)))
main_city_quantity

,排名,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
568,1,盈都·五角广场,钱塘新区,579,27002,39246,14535
529,2,溪悦云和园,西湖区,542,66379,214456,32308
566,3,湖印光年,钱塘新区,490,58380,173970,29800
551,4,东原钱塘印未来,钱塘新区,364,34818,62621,17985
363,5,保利天汇,滨江区,317,34876,179365,51429
283,6,河语光年府,拱墅区,316,40901,155048,37908
557,7,兴耀·岚漫之城,钱塘新区,308,33980,67056,19734
26,8,锦尚和品府,下城区,307,44857,232561,51844
572,9,融创·杭望云潮城,钱塘新区,296,35269,106921,30316
286,10,潮和阅江台轩,拱墅区,277,27994,90542,32343


In [91]:
# 主城区面积
main_city_area = merge_data[merge_data['所属区域'].isin(main_city)].sort_values('签约面积（㎡）', ascending=False).head(11)
main_city_area.insert(0, '排名', list(range(1,12,1)))
main_city_area

,排名,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
529,1,溪悦云和园,西湖区,542,66379,214456,32308
566,2,湖印光年,钱塘新区,490,58380,173970,29800
26,3,锦尚和品府,下城区,307,44857,232561,51844
283,4,河语光年府,拱墅区,316,40901,155048,37908
572,5,融创·杭望云潮城,钱塘新区,296,35269,106921,30316
363,6,保利天汇,滨江区,317,34876,179365,51429
551,7,东原钱塘印未来,钱塘新区,364,34818,62621,17985
557,8,兴耀·岚漫之城,钱塘新区,308,33980,67056,19734
507,9,中杭府,西湖区,151,30882,164088,53134
7,10,潮听明月公寓,上城区,241,30505,130489,42776


In [92]:
# 萧山
xiaoshan_quantity = merge_data[merge_data['所属区域'] =='萧山区'].sort_values(by=['签约套数', '签约面积（㎡）'], ascending=False).head(11)
xiaoshan_quantity.insert(0, '排名', list(range(1,12,1)))
xiaoshan_quantity

,排名,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
477,1,绿城·桂冠东方,萧山区,1162,162629,754238,46378
413,2,前湾国际中心,萧山区,931,130647,385604,29515
460,3,滨江·望涛月明,萧山区,757,91957,343850,37392
461,4,滨江·滨涛映月,萧山区,566,66296,247950,37400
481,5,花屿观澜里,萧山区,459,52484,197530,37636
399,6,亚奥城,萧山区,414,60084,277538,46192
487,7,融信·滨望雅庭,萧山区,403,45245,128958,28502
383,8,万科·日耀之城,萧山区,289,43289,200078,46219
395,9,之彩城,萧山区,226,24171,40980,16954
425,10,尚品华章府,萧山区,207,20305,34891,17184


In [93]:
yuhang_quantity = merge_data[merge_data['所属区域'] =='余杭区'].sort_values(by=['签约套数', '签约面积（㎡）'], ascending=False).head(11)
yuhang_quantity.insert(0, '排名', list(range(1,12,1)))
yuhang_quantity

,排名,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
122,1,云泓之城,余杭区,211,37294,119535,32052
134,2,启航城,余杭区,130,13748,27350,19894
152,3,新城英冠·香悦和鸣,余杭区,116,13927,39257,28187
131,4,华景川科大·湖沁宸园,余杭区,104,10357,20300,19600
141,5,天樾四季,余杭区,100,11366,26962,23722
115,6,中旅时光里,余杭区,100,11032,33416,30289
194,7,西房·良语云缦,余杭区,75,8617,24849,28837
123,8,保利融信·和光尘樾,余杭区,67,8119,16689,20555
160,9,星创城|映月璟园,余杭区,60,6774,19435,28690
132,10,华润置地杭珹未来中心,余杭区,51,6481,23339,36013


In [94]:
fuyang_quantity = merge_data[merge_data['所属区域'] =='富阳区'].sort_values(by=['签约套数', '签约面积（㎡）'], ascending=False).head(11)
fuyang_quantity.insert(0, '排名', list(range(1,12,1)))
fuyang_quantity

,排名,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
243,1,秦望府,富阳区,121,18590,57720,31050
247,2,绿城·桂语新月,富阳区,94,9864,21525,21821
239,3,瑞源邸,富阳区,89,8850,18036,20379
252,4,融创·云辰富源里,富阳区,80,8661,22364,25822
229,5,招商德信·云望之宸,富阳区,57,6184,14252,23046
216,6,九龙仓·雍景山,富阳区,56,8151,19165,23512
253,7,观宇锦云里,富阳区,52,5626,11989,21310
228,8,悦望名邸,富阳区,46,6216,16498,26541
213,9,中交保利江语云城,富阳区,40,4095,6661,16267
234,10,泰禾野风·大城小院,富阳区,38,4562,10831,23744


In [95]:
linan_quantity = merge_data[merge_data['所属区域'] =='临安区'].sort_values(by=['签约套数', '签约面积（㎡）'], ascending=False).head(11)
linan_quantity.insert(0, '排名', list(range(1,12,1)))
linan_quantity

,排名,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
31,1,中天·珺府（南区）,临安区,137,13648,32601,23887
107,2,颐和山庄,临安区,131,13679,32292,23607
90,3,芳泽锦安里,临安区,122,13551,10107,7458
82,4,滨虹商务中心,临安区,101,10426,11441,10974
98,5,越秀星悦城,临安区,88,9673,19778,20446
101,6,银城青山湖畔,临安区,79,9136,24704,27041
102,7,锦晟里,临安区,71,5787,11248,19437
52,8,同人山庄,临安区,65,8970,35110,39142
54,9,国风（明煦）源赋,临安区,63,5817,11494,19761
58,10,如沐清晖城,临安区,61,10634,34450,32395


### 导出数据

In [96]:
file_path1 = '排行结果/' + month + '/'
if not os.path.exists(file_path1):
    os.mkdir(file_path1)
file_name1 = month + '杭州楼市成交排行榜.xlsx'
writer = pd.ExcelWriter(file_path1 + file_name1)
data_list = [
    main_city_quantity, main_city_area, xiaoshan_quantity, yuhang_quantity,
    linan_quantity, fuyang_quantity
]
row = 2
for d in data_list:
    d.to_excel(writer, startrow=row, sheet_name="新房排行", index=False)
    row += 15

# 保存
writer.save()
writer.close()

rb = xlrd.open_workbook(file_path1+file_name1)   
# rb = openpyxl.load_workbook(file_path1+file_name1)   
wb = copy.copy(rb) 
# sheets = wb.worksheets
ws = wb.get_sheet(0)    
ws.write(1, 0, '主城套数') 
ws.write(16, 0, '主城面积') 
ws.write(31, 0, '萧山套数') 
ws.write(46, 0, '余杭套数') 
ws.write(61, 0, '临安套数') 
ws.write(76, 0, '富阳套数') 
wb.save(file_path1+file_name1)  

H:\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## 住宅非住宅成交榜

### 计算排行

In [97]:
house_data = house_data.drop(0)[['项目名称', '城区', '成交套数', '成交面积(㎡)', '预测成交总价(万元)', '预测成交均价(元/㎡)']]
house_data.columns = ['项目名称', '所属区域', '签约套数', '签约面积（㎡）', '签约金额（万元）', '签约均价（元/㎡）']

house_data= house_data.replace({'祥生云湖城':'祥生银湖宸语'})
calc_data_house = house_data.copy()
house_data.head()

,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
1,绿城·桂冠东方,萧山区,1162,162629.46,754238,46378
2,前湾国际中心,萧山区,931,130646.62,385604,29515
3,滨江·望涛月明,萧山区,754,91814.92,343210,37381
4,滨江·滨涛映月,萧山区,558,65929.30,246443,37380
5,溪悦云和园,西湖区,542,66379.25,214456,32308


In [98]:
non_house_data = non_house_data.drop(0)[['项目名称', '城区', '成交套数', '成交面积(㎡)', '预测成交总价(万元)', '预测成交均价(元/㎡)']]
non_house_data.columns = ['项目名称', '所属区域', '签约套数', '签约面积（㎡）', '签约金额（万元）', '签约均价（元/㎡）']
calc_data_non_house = non_house_data.copy()
non_house_data.head()

,项目名称,所属区域,签约套数,签约面积（㎡）,签约金额（万元）,签约均价（元/㎡）
1,盈都·五角广场,钱塘新区,579,27001.84,39246,14535
2,杭州世纪中心,萧山区,188,61149.21,164716,26937
3,西溪宸悦,西湖区,183,11431.27,32279,28237
4,融创运河印,拱墅区,179,12704.45,27767,21856
5,万象世界中心,萧山区,170,75011.42,185944,24789


In [99]:
# 住宅数据排行
merge_data_house = calc_data_house.groupby(
    ['所属区域', '项目名称'])[['签约套数', '签约面积（㎡）', '签约金额（万元）']].sum()
merge_data_house.loc[:, ['签约面积（㎡）', '签约金额（万元）']] = round(merge_data_house.loc[:, ['签约面积（㎡）', '签约金额（万元）']], 0)
merge_data_house.loc[:, '签约均价'] =(merge_data_house.loc[:, '签约金额（万元）']*10000/merge_data_house.loc[:, '签约面积（㎡）']).round().astype(int)
merge_data_house.loc[:, '均价'] = round(merge_data_house.loc[:, '签约均价'], -2)
merge_data_house = merge_data_house.reset_index()
merge_data_house.rename(columns={'签约均价': '签约均价（元/㎡）'}, inplace=True)
merge_data_house = merge_data_house[['项目名称', '所属区域', '签约套数', '签约面积（㎡）', '签约金额（万元）', '签约均价（元/㎡）', '均价']]
merge_data_house.columns = ['楼盘名称', '城区', '套数', '签约面积', '签约金额', '签约均价', '均价']
merge_data_house

,楼盘名称,城区,套数,签约面积,签约金额,签约均价,均价
0,世茂风颂府,上城区,70,9347.0,37945,40596,40600
1,招商德信·花芸春境,上城区,6,614.0,1944,31661,31700
2,滨江金地·御品,上城区,16,2312.0,15553,67271,67300
3,潮听明月公寓,上城区,241,30505.0,130489,42776,42800
4,融创滨江·杭源御潮府,上城区,1,188.0,1453,77287,77300
...,...,...,...,...,...,...,...
326,融创·杭望云潮城,钱塘新区,296,35269.0,106921,30316,30300
327,融创杭臻源,钱塘新区,87,10650.0,40722,38237,38200
328,融创正荣·杭曜之城,钱塘新区,240,28466.0,85350,29983,30000
329,金成祥新·潮起钱塘,钱塘新区,1,100.0,211,21100,21100


In [100]:
merge_data_house[merge_data_house['均价']<0]

,楼盘名称,城区,套数,签约面积,签约金额,签约均价,均价


In [101]:
# 住宅成交榜
house_rank = merge_data_house.sort_values('套数', ascending=False).head(10)
house_rank = house_rank[['楼盘名称', '城区', '套数', '均价']]
house_rank

,楼盘名称,城区,套数,均价
286,绿城·桂冠东方,萧山区,1162,46400
253,前湾国际中心,萧山区,931,29500
277,滨江·望涛月明,萧山区,754,37400
278,滨江·滨涛映月,萧山区,558,37400
306,溪悦云和园,西湖区,542,32300
322,湖印光年,钱塘新区,490,29800
288,花屿观澜里,萧山区,459,37600
246,亚奥城,萧山区,414,46200
292,融信·滨望雅庭,萧山区,403,28500
311,东原钱塘印未来,钱塘新区,364,18000


In [102]:
# 住宅均价35000以上
house_rank1 = merge_data_house[merge_data_house['签约均价']>35000].sort_values('套数', ascending=False).head(10)
house_rank1 = house_rank1[['楼盘名称', '城区', '套数', '均价']]
house_rank1

,楼盘名称,城区,套数,均价
286,绿城·桂冠东方,萧山区,1162,46400
277,滨江·望涛月明,萧山区,754,37400
278,滨江·滨涛映月,萧山区,558,37400
288,花屿观澜里,萧山区,459,37600
246,亚奥城,萧山区,414,46200
230,保利天汇,滨江区,317,51400
193,河语光年府,拱墅区,316,37900
11,锦尚和品府,下城区,304,51800
237,万科·日耀之城,萧山区,289,46200
197,禹洲宋都望林府,拱墅区,269,42800


In [103]:
# 住宅均价20000以上35000以下
house_rank2 = merge_data_house[(merge_data_house['签约均价'] > 20000) & (
    merge_data_house['签约均价'] <= 35000)].sort_values('套数', ascending=False).head(10)
house_rank2 = house_rank2[['楼盘名称', '城区', '套数', '均价']]
house_rank2

,楼盘名称,城区,套数,均价
253,前湾国际中心,萧山区,931,29500
306,溪悦云和园,西湖区,542,32300
322,湖印光年,钱塘新区,490,29800
292,融信·滨望雅庭,萧山区,403,28500
326,融创·杭望云潮城,钱塘新区,296,30300
196,潮和阅江台轩,拱墅区,277,32300
328,融创正荣·杭曜之城,钱塘新区,240,30000
296,赞成漫悦轩,萧山区,204,28300
303,古墩彩虹轩,西湖区,201,28100
259,宁望府,萧山区,186,33500


In [104]:
# 住宅均价20000以下
house_rank3 = merge_data_house[merge_data_house['签约均价']<=20000].sort_values('套数', ascending=False).head(10)
house_rank3 = house_rank3[['楼盘名称', '城区', '套数', '均价']]
house_rank3

,楼盘名称,城区,套数,均价
311,东原钱塘印未来,钱塘新区,364,18000
315,兴耀·岚漫之城,钱塘新区,308,19700
243,之彩城,萧山区,226,17000
260,尚品华章府,萧山区,207,17200
294,融创·森与海之城,萧山区,177,13400
95,启航城,余杭区,130,19900
65,芳泽锦安里,临安区,122,7500
256,君铂府,萧山区,113,16000
93,华景川科大·湖沁宸园,余杭区,104,19600
310,三迪·雅颂美筑,钱塘新区,90,14400


In [105]:
#非住宅数据排行
merge_data_non_house = calc_data_non_house.groupby(
    ['所属区域', '项目名称'])[['签约套数', '签约面积（㎡）', '签约金额（万元）']].sum()
merge_data_non_house.loc[:, ['签约面积（㎡）', '签约金额（万元）']] = merge_data_non_house.loc[:, ['签约面积（㎡）', '签约金额（万元）']].round()
merge_data_non_house.loc[:, '签约均价'] = (merge_data_non_house.loc[:,
                                                     '签约金额（万元）']*10000/merge_data_non_house.loc[:, '签约面积（㎡）']).astype(int)
merge_data_non_house.loc[:, '均价'] = round(merge_data_non_house.loc[:, '签约均价'], -2)
merge_data_non_house = merge_data_non_house.reset_index()
merge_data_non_house.rename(columns={'签约均价': '签约均价（元/㎡）'}, inplace=True)
merge_data_non_house = merge_data_non_house[[
    '项目名称', '所属区域', '签约套数', '签约面积（㎡）', '签约金额（万元）', '签约均价（元/㎡）', '均价']]
merge_data_non_house.columns = ['楼盘名称', '城区', '套数', '签约面积', '签约金额', '签约均价', '均价']
merge_data_non_house

,楼盘名称,城区,套数,签约面积,签约金额,签约均价,均价
0,九天泽一国际,上城区,3,118.0,791,67033,67000
1,信达东莱府,上城区,1,38.0,154,40526,40500
2,招商越秀·公园1872,上城区,51,1993.0,6850,34370,34400
3,杭州钱潮中心,上城区,3,1026.0,6362,62007,62000
4,融信·江河盛会,上城区,28,9708.0,32118,33084,33100
...,...,...,...,...,...,...,...
309,盈都·五角广场,钱塘新区,579,27002.0,39246,14534,14500
310,祥生湛景·江山云樾,钱塘新区,39,2175.0,4750,21839,21800
311,蓬和中天纪中心,钱塘新区,1,315.0,428,13587,13600
312,融创·云帆未来社区,钱塘新区,50,2581.0,4907,19012,19000


In [106]:
merge_data_non_house[merge_data_non_house.loc[:, '均价']<0]

,楼盘名称,城区,套数,签约面积,签约金额,签约均价,均价


In [107]:
# 非住宅成交榜
non_house_rank = merge_data_non_house.sort_values('套数', ascending=False).head(10)
non_house_rank = non_house_rank[['楼盘名称', '城区', '套数', '均价']]
non_house_rank

,楼盘名称,城区,套数,均价
309,盈都·五角广场,钱塘新区,579,14500
233,杭州世纪中心,萧山区,188,26900
294,西溪宸悦,西湖区,183,28200
131,融创运河印,拱墅区,179,21900
195,万象世界中心,萧山区,170,24800
199,中量·畅享幸福里,萧山区,142,9300
270,中融蓝城Co.C理想城,西湖区,110,25900
255,萧政储出[2010]52号地块,萧山区,105,7500
128,苏荷汇,拱墅区,103,31600
34,滨虹商务中心,临安区,101,11000


### 导出数据

In [108]:
file_path2 = './排行结果/' + month + '/'
if not os.path.exists(file_path2):
    os.mkdir(file_path2)
file_name2 = month + '杭州新房成交榜(住宅非住宅均价).xlsx'
writer = pd.ExcelWriter(file_path2 + file_name2)

house_rank.to_excel(writer, startrow=1, startcol=0, sheet_name='Sheet1', index=False)
non_house_rank.to_excel(writer, startrow=1, startcol=5, index=False)
house_rank1.to_excel(writer, startrow=15, startcol=0, index=False)
house_rank2.to_excel(writer, startrow=15, startcol=5, index=False)
house_rank3.to_excel(writer, startrow=15, startcol=10, index=False)

# 保存
writer.save()
writer.close()

rb = xlrd.open_workbook(file_path2 + file_name2)   
wb = copy.copy(rb)                         
ws = wb.get_sheet(0)                 
ws.write(0, 0, month + '杭州市区住宅成交排名前十数据') 
ws.write(0, 5, month + '杭州市区非住宅项目成交排名前十数据') 
ws.write(14, 0, month + '杭州市区住宅均价在35000元/㎡以上成交排名前十数据') 
ws.write(14, 5, month + '杭州市区住宅均价在20000-35000元/㎡成交排名前十数据') 
ws.write(14, 10, month + '杭州市区住宅均价在20000元/㎡以下成交排名前十数据') 
wb.save(file_path2 + file_name2)  

H:\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## 房企排行榜

In [77]:
all_data = all_data.drop(0)[['项目名称', '城区', '成交套数', '成交面积(㎡)', '预测成交总价(万元)', '预测成交均价(元/㎡)']]
all_data.columns = ['楼盘名称', '所属区域', '签约套数', '签约面积（㎡）', '销售金额', '签约均价（元/㎡）']
calc_data_all = all_data.copy()
all_data.head()

,楼盘名称,所属区域,签约套数,签约面积（㎡）,销售金额,签约均价（元/㎡）
1,绿城·桂冠东方,萧山区,1082,152829.28,709608,46431
2,前湾国际中心,萧山区,931,130646.62,385604,29515
3,滨江·望涛月明,萧山区,755,91741.03,343019,37390
4,盈都·五角广场,钱塘新区,579,27001.84,39246,14535
5,滨江·滨涛映月,萧山区,566,66296.39,247950,37400


In [78]:
file_path3 = '排行结果/' + month + '/'
if not os.path.exists(file_path3):
    os.mkdir(file_path3)

In [79]:
for file in os.listdir(block_dir):
    if '房企' in file:
        print(file)
        group_data_file_path = block_dir + file

3月份房企榜单数据需求.xls


In [80]:
table=open_workbook(group_data_file_path)   #打开文件
get_sheets = table.sheet_names()    #获取excel的sheet页的名称，全部打印出来
group_list = [[] for _ in range(len(get_sheets))]
df_list = [pd.DataFrame() for _ in range(len(get_sheets))]
for i in get_sheets:
    get_each_sheet = table.sheet_by_name(i)  #获取到每个sheet页的名称，单独打印
    print(get_each_sheet.name)
    count_rows = get_each_sheet.nrows   
    for j in range(count_rows):  
        col_values = get_each_sheet.row_values(j, start_colx=0, end_colx=None)
        group_list[get_sheets.index(i)].append(col_values)
    df_list[get_sheets.index(i)] = pd.DataFrame(group_list[get_sheets.index(i)])
    df_list[get_sheets.index(i)].columns = ['楼盘名称', '销售金额']
    df_list[get_sheets.index(i)].drop(index = 0, inplace=True)
    df_list[get_sheets.index(i)].drop('销售金额', axis=1, inplace=True)
    print(df_list[get_sheets.index(i)], '\n')

联合操盘
        楼盘名称
1     绿城·运河郡
2       桃李春风
3   金茂万科·新都会
4  滨江新希望·悦潮府
5        悦融湾 

滨江
               楼盘名称
1           滨江·望涛月明
2           滨江·滨涛映月
3                望品
4   滨融府新象·light art
5               宁望府
..              ...
74             沁语晓庭
75            庆丰拥涛府
76            仁恒滨江园
77         新希望滨江锦粼府
78           新越锦宸名邸

[78 rows x 1 columns] 

万科
         楼盘名称
1     万科·日耀之城
2        湖印光年
3       河语光年府
4        云泓之城
5       和家园臻园
..        ...
80       金色家园
81  金沙印象城·JIC
82       君望花苑
83  良渚文化村·秋荷坊
84  良渚文化村·阳光天

[84 rows x 1 columns] 

绿城
       楼盘名称
1   绿城·桂冠东方
2    潮听明月公寓
3   绿城·桂语新月
4     山澜桂语轩
5   绿城·江上臻园
..      ...
74    绿城茗春苑
75  赛丽绿城·慧园
76  赛丽绿城·丽园
77    溪上玫瑰园
78   香岸云栖燕庐

[78 rows x 1 columns] 

华润
            楼盘名称
1            亚奥城
2         万象世界中心
3      城北万象城·幸福里
4          亚奥城南区
5       杭州中央车站广场
6     华润置地杭珹未来中心
7         万象城·悦玺
8           之江九里
9            御清城
10            悦府
11         华润万象汇
12         华润·翠庭
13  杭政储出[2021]44 

融创
            楼盘名称
1       融创·杭望云潮城

In [81]:
writer = pd.ExcelWriter(file_path3 + '房企榜单数据需求.xlsx')

In [82]:
a=0
for i in range(len(df_list)):
    df_list[i] = pd.merge(df_list[i], calc_data_all, on='楼盘名称', how='left')
    df_list[i] = df_list[i][['楼盘名称', '销售金额']]
    print(get_sheets[i])
    df_list[i].to_excel(writer, get_sheets[a], index=False)
    a = a+1
#     print(df)

联合操盘
滨江
万科
绿城
华润
融创
保利
仁恒
龙湖
招商
杭州地铁


In [83]:
writer.save()
writer.close()

H:\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
